In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
from scipy.stats import skew, kurtosis
from statsmodels.tsa import stattools
from sklearn.model_selection import train_test_split

In [2]:
########################################################################################
#                                                                                      #
# Extract Data Mentah Menjadi Data Klasifikasi                                         #
# https://github.com/nlathia/pydata_2016/blob/master/Learn/Accelerometer-Explore.ipynb #
#                                                                                      #
########################################################################################

In [3]:
# https://www.sciencedirect.com/science/article/pii/S0893608014000203
# Page 2 Sub-title 3.1 Acceleration synthesization

def magnitude(activity):    
    x2 = activity['xAxis'] * activity['xAxis']
    y2 = activity['yAxis'] * activity['yAxis']
    z2 = activity['zAxis'] * activity['zAxis']
    m2 = x2 + y2 + z2
    m = m2.apply(lambda x: math.sqrt(x))
    return m

In [4]:
def jitter(axis, start, end):
    j = float(0)
    for i in range(start, min(end, axis.count())):
        if start != 0:
            j += abs(axis[i] - axis[i-1])
    return j / (end-start)

In [5]:
def mean_crossing_rate(axis, start, end):
    cr = 0
    m = axis.mean()
    for i in range(start, min(end, axis.count())):
        if start != 0:
            p = axis[i-1] > m
            c = axis[i] > m
            if p != c:
                cr += 1
    return float(cr) / (end-start-1)

In [6]:
def hidden_window_summary(axis, start, end):
    acf = stattools.acf(axis[start:end])                                            # https://en.wikipedia.org/wiki/Autocorrelation
    acv = stattools.acovf(axis[start:end])                                          # https://en.wikipedia.org/wiki/Autocovariance
    sqd_error = (axis[start:end] - axis[start:end].mean()) ** 2
    return [
        jitter(axis, start, end),             # Jitter                              # https://en.wikipedia.org/wiki/Jitter
        mean_crossing_rate(axis, start, end), # Mean Crossing Rate
        axis[start:end].mean(),               # Mean
        axis[start:end].std(),                # Standard Deviation                  # https://en.wikipedia.org/wiki/Standard_deviation
        axis[start:end].var(),                # Variance                            # https://en.wikipedia.org/wiki/Variance
        axis[start:end].min(),                # Minimum
        axis[start:end].max(),                # Maximum
        acf.mean(),                           # mean auto correlation
        acf.std(),                            # standard deviation auto correlation
        acv.mean(),                           # mean auto covariance
        acv.std(),                            # standard deviation auto covariance
        skew(axis[start:end]),                # skewness                            # https://en.wikipedia.org/wiki/Skewness
        kurtosis(axis[start:end]),            # kurtosis                            # https://en.wikipedia.org/wiki/Kurtosis
        math.sqrt(sqd_error.mean())           # Square Root Deviation
    ]

In [7]:
def hidden_windows(df, size=100):
    start = 0
    while start < df.count():
        yield start, start + size
        start += (size / 2)

In [8]:
def features(activity):
    for (start, end) in hidden_windows(activity['timestamp']):
        features = []
        for axis in ['xAxis', 'yAxis', 'zAxis', 'magnitude']:
            features += hidden_window_summary(activity[axis], int(start), int(end))
        yield features

In [9]:
# 3 fungsi dibawah hanya untuk menampilkan grafik.
# Tidak berpengaruh terhadap data.

def plot_axis(ax, x, y, title):
    ax.plot(x, y)
    ax.set_title(title)
    ax.xaxis.set_visible(False)
    ax.set_ylim([min(y) - np.std(y), max(y) + np.std(y)])
    ax.set_xlim([min(x), max(x)])
    ax.grid(True)

def plot_activity(activity):
    fig, (ax0, ax1, ax2) = plt.subplots(nrows=3, figsize=(15, 10), sharex=True)
    plot_axis(ax0, activity['timestamp'], activity['xAxis'], 'x Axis')
    plot_axis(ax1, activity['timestamp'], activity['yAxis'], 'y Axis')
    plot_axis(ax2, activity['timestamp'], activity['zAxis'], 'z Axis')
    plt.subplots_adjust(hspace=0.2)
    plt.show()

def plot_magnitudes(activities, titles, hidden_inputdow=False):
    fig, axs = plt.subplots(nrows=len(activities), figsize=(15, 15))
    for i in range(0, len(activities)):
        plot_axis(axs[i], activities[i]['timestamp'], activities[i]['magnitude'], titles[i])
        if hidden_inputdow == True:
            for (start, end) in hidden_inputdows(activities[i]['timestamp']):
                axs[i].axvline(activities[i]['timestamp'][start], color='r')
    plt.subplots_adjust(hspace=0.2)
    plt.show()

In [10]:
# Contoh isi dari file (Train_1462518004872.csv)
#
# timestamp         xAxis       yAxis       zAxis
# 1462518004907     0.9163807   5.911284    7.6027875
# 1462518004907     0.9163807   5.911284    7.6027875
# 1462518004908     0.9163807   5.911284    7.6027875
# 1462518004908     0.9163807   5.911284    7.6027875
# 1462518004926     0.8822633   5.9561753   7.6943655
# ...
# 1462518115208     2.926313    4.7734394   7.647679
# 1462518115228     3.0298622   4.8895583   7.8547773
# 1462518115250     3.0310593   4.819528    7.8446016
# 1462518115275     2.9005754   4.6124296   7.752425
# 1462518115293     2.7491422   4.648941    7.7159133

COLUMNS = ['timestamp', 'xAxis', 'yAxis', 'zAxis']
data_mentah = pd.read_csv('Train_1462518004872.csv', header=None, names=COLUMNS)

In [11]:
# Pastikan sebelum meng-ekstrak, terlebih dahulu sudah ada data
# timestamp, xAxis, yAxis, zAxis, magnitude

data_mentah['magnitude'] = magnitude(data_mentah)     # tambah magnitude
data_test = list(features(data_mentah))               # hasil ekstrak

In [12]:
#############################################################################
#                                                                           #
# Extreme Learning Machine                                                  #
# https://www.kaggle.com/robertbm/extreme-learning-machine-example/notebook #
#                                                                           #
#############################################################################

In [13]:
# Contoh isi dari file (Dataset_Erick.csv)
#
# 0, 0.0, 0.0, 0.24989480049999999, 0.01015862876539376, 0.00010319773839308553, 0.22565351, 0.27832594999999999, 0.017607833285933207, 0.19224009571074763, 5.1082880504577306e-07, 1.2934064787687153e-05, 0.1654739742918587, -0.046037477003179283, 0.010107707999796723, 0.0, 0.0, -0.048207251090000007, 0.012737698407271909, 0.00016224896071461735, -0.083797060000000007, -0.018555064, 0.02854321243166958, 0.22894512013958773, 8.0313235553735594e-07, 2.7058941102581519e-05, -0.15416106399941704, -0.36436713079124505, 0.012673849892888552, 0.0, 0.0, 9.610876339999999, 0.012443421923289892, 0.00015483874916101152, 9.5815950000000001, 9.6402529999999995, 0.027165215652069495, 0.173794698641765, 7.6645180834700735e-07, 1.8505989232888895e-05, 0.38774998560198987, 0.05882933975061411, 0.012381048488290537, 0.0, 0.0, 9.6142590200783804, 0.012510625047233163, 0.00015651573907245778, 9.5849311096040335, 9.6436152553621586, 0.026969040805525996, 0.1738030968233161, 7.7475290840866495e-07, 1.8710167261818296e-05, 0.37677552479231574, 0.03923906701560176, 0.01244791475234841
# 0, 0.0110193128, 0.4444444444444444, 0.24805126300000002, 0.010428860696702456, 0.00010876113543122523, 0.21787234999999999, 0.27593174999999998, 0.014541107144808255, 0.19032942260814673, 5.3836762038456478e-07, 1.3751043293263152e-05, -0.03360925874166811, -0.0007566405332890547, 0.010376585376553936, 0.013407529280000002, 0.40404040404040403, -0.048566381350000007, 0.015135094566564238, 0.00022907108753884235, -0.083797060000000007, -0.018555064, 0.02621916570145957, 0.26901648108784099, 1.1339018833172687e-06, 4.3524356337674224e-05, -0.27295521736311906, -0.7143031987783468, 0.015059228953152082, 0.013748760000000075, 0.5858585858585859, 9.6116783900000016, 0.011375358796334953, 0.000129398787745355, 9.5881799999999995, 9.6396540000000002, 0.011320914657461925, 0.18368911348519995, 6.405239993395075e-07, 1.5692264589393077e-05, 0.4958786735952581, -0.11220112352915956, 0.01131833909493356, 0.013780593563835576, 0.6161616161616161, 9.6150186451639339, 0.011397639170174605, 0.00012990617865349843, 9.5914202146449465, 9.6430042967070406, 0.011430466403016036, 0.1837229736584399, 6.4303558433481724e-07, 1.5755919157526435e-05, 0.4978057475510153, -0.09000285807020125, 0.011340507786998051
# 0, 0.0092116902999999951, 0.37373737373737376, 0.24554333720000002, 0.0099975288652464772, 9.9950583411436501e-05, 0.21787234999999999, 0.27114335000000001, 0.029772714593831129, 0.21869196734134372, 4.9475538788661031e-07, 1.5304468738391751e-05, 0.09325687572132318, -0.2156749208008515, 0.009947415623031046, 0.013150152549999999, 0.35353535353535354, -0.04917091719999999, 0.015474249958621447, 0.00023945241178189585, -0.087388359999999998, -0.016759411999999998, 0.026880365008253027, 0.27396024700687027, 1.1852894383203857e-06, 4.5226384606928164e-05, -0.1258665891421242, -0.5437636067462637, 0.015396684307475973, 0.010253150000000079, 0.5151515151515151, 9.6100203799999981, 0.00868443604965842, 7.5419429500606756e-05, 9.5911720000000003, 9.6288804999999993, 0.012175486889396863, 0.18423966147863782, 3.7332617602800346e-07, 9.5206151260687985e-06, -0.07253614135010605, -0.4963659408810237, 0.00864090476776597, 0.010243855518327454, 0.5151515151515151, 9.6133000213403577, 0.0086682664543859082, 7.5138843324232032e-05, 9.5948843744597063, 9.6322125622293537, 0.012502969901594154, 0.18438025835849609, 3.7193727445494832e-07, 9.491125193029967e-06, -0.07188154716938473, -0.49103354815993994, 0.008624816223606722
# 0, 0.0083677324999999973, 0.35353535353535354, 0.24606407550000001, 0.0093857741346657364, 8.809275610696035e-05, 0.22146365000000001, 0.26934770000000002, 0.02913834338547279, 0.20896841524066978, 4.3605914272945362e-07, 1.3316071246929901e-05, -0.1900682425457343, -0.4486780372084711, 0.009338727351512664, 0.012048820069999997, 0.36363636363636365, -0.047770309410000003, 0.013487608742589164, 0.00018191558959316766, -0.087388359999999998, -0.016759411999999998, 0.015313497467495968, 0.23303459584768932, 9.0048216848617934e-07, 2.9408399498669667e-05, -0.11839711259146216, -0.01862330894946984, 0.013420001255485633, 0.010235210000000041, 0.48484848484848486, 9.6108343999999981, 0.0088002727876108702, 7.7444801136364387e-05, 9.5899750000000008, 9.630077, 0.024242048937504979, 0.16912843851223389, 3.8335176562500359e-07, 9.0991008275553847e-06, -0.19889304524533935, -0.5204274068206685, 0.008756160866784069, 0.010172701251532832, 0.46464646464646464, 9.6141164575532621, 0.0087683518945926315, 7.688399494740619e-05, 9.593598417969682, 9.633162412863534, 0.025153439672230951, 0.16864004477556579, 3.8057577498966081e-07, 9.0276175788816705e-06, -0.1985859959099306, -0.5271861510161342, 0.008724399979249698
# 0, 0.010606313999999999, 0.37373737373737376, 0.24699781529999998, 0.012171632439769342, 0.00014814863624884538, 0.21547815000000001, 0.27653030000000001, 0.018820047412159063, 0.25489944491120603, 7.3333574943178448e-07, 2.7324517729387446e-05, -0.11279270159450296, 0.11531882880259392, 0.012110621366649895, 0.010797849529999997, 0.4444444444444444, -0.048853685600000002, 0.010720294397164423, 0.00011492471196187491, -0.072424600000000006, -0.023343466, 0.013788728229595973, 0.2315660767995098, 5.6887732421128091e-07, 1.7664525009113588e-05, -0.06259866444764854, -0.4230751747002004, 0.010666558247263086, 0.010630339999999966, 0.4444444444444444, 9.6108283899999982, 0.0097675060924248529, 9.5404175265556622e-05, 9.5899750000000008, 9.6402529999999995, 0.016996846536664716, 0.17697649697750814, 4.7225066756450547e-07, 1.1252746039825579e-05, 0.13342891537303464, -0.3611438403247771, 0.0097185458538251, 0.010548133778030575, 0.4444444444444444, 9.6141394702243499, 0.0097486187663761107, 9.5035567852140486e-05, 9.593598417969682, 9.6437059571336263, 0.017250691369539568, 0.17686401547241987, 4.7042606086809539e-07, 1.1194186821258836e-05, 0.13796571690888706, -0.35707260554649745, 0.009699753201686066
# ...
# 4, 0.062309105999999975, 0.0, 2.8826008860000001, 0.11919706642251285, 0.014207940643732939, 2.6491842000000001, 3.1711202000000003, 0.073988217442263693, 0.25179747290858984, 7.0329306186478024e-05, 0.0025330808957646556, 0.14158443120191827, -0.33438919532107825, 0.1185995836303636, 0.054617700999999991, 0.0, 5.0697340909999999, 0.12864434744741218, 0.016549368130170508, 4.7896004000000003, 5.3366755999999995, 0.18490791417457353, 0.24510443526020367, 8.1919372244343856e-05, 0.0038380307150959253, 0.023057088630927126, -0.45220158375784436, 0.1279995095649542, 0.087669626000000098, 0.0, 7.7477324649999995, 0.14274496976706494, 0.020376126393800285, 7.4962453999999994, 8.1001829999999995, 0.097719036617802166, 0.20402036887948166, 0.00010086182564931139, 0.0032170607170564488, 0.36460386579593407, -0.5502546228321186, 0.1420294516283939, 0.0885504312225565, 0.30303030303030304, 9.6992488669910806, 0.11952552501513394, 0.014286351130143409, 9.4379883641321616, 9.9369470512850491, 0.0065864035047879686, 0.21501779207894742, 7.0717438094209883e-05, 0.0020441538483091733, 0.12830983827006917, -0.7346030899708968, 0.1189263958036313
# 4, 0.076093709000000023, 0.0, 2.903155097, 0.14390283643378748, 0.020708026333689396, 2.4552537999999995, 3.1711202000000003, 0.030088588005791575, 0.24103392841989513, 0.00010250473035176264, 0.0035289842711673848, -0.5870230818342119, 0.5385102561772079, 0.14318151441562735, 0.053205146000000016, 0.0, 5.1399799459999995, 0.096130223596812697, 0.0092410198887732057, 4.8895583, 5.3366755999999995, -0.0027570431529685916, 0.2645772969788498, 4.5743048449427385e-05, 0.0015776751290803945, -0.29546823248337173, 0.07411629031224543, 0.09564836480507899, 0.10109516000000021, 0.0, 7.6736917680000012, 0.17836501097704452, 0.03181407714084121, 7.156269, 8.292916, 0.0060278651699514959, 0.26057315712265888, 0.00015747968184716399, 0.0054775565796267691, 0.578979909652004, 1.01903490194334, 0.17747094514154366, 0.099008170768498771, 0.2222222222222222, 9.6832732127231509, 0.17000918774391455, 0.028903123917345581, 9.1731254285963111, 10.179173916834701, 0.010738497238648267, 0.25358565994130894, 0.00014307046339086054, 0.0048369554574876957, 0.2998335803640513, 0.32245410057143253, 0.1691570059978957
# 4, 0.067797800000000019, 0.0, 2.8340883649999999, 0.15252880021908394, 0.023265034896273219, 2.4552537999999995, 3.1657330000000004, 0.083450696006850095, 0.23808108808448422, 0.0001151619227365526, 0.0040624728669893459, -0.19146170001768603, -0.4355951527116191, 0.151764240014934, 0.045058877000000018, 0.0, 5.1064371620000006, 0.11131186570058291, 0.012390331445744605, 4.8787846999999998, 5.3432599999999999, 0.043013259599741831, 0.30429208289132087, 6.133214065643576e-05, 0.0025535687756342382, -0.4751778791708928, -0.6361503157968724, 0.11075390797297927, 0.097318353000000121, 0.0, 7.7546037830000003, 0.21580756704541351, 0.046572905994060651, 7.156269, 8.292916, 0.09026583542870692, 0.22343016085477202, 0.00023053588467060026, 0.0075927353476262878, -0.07891053472424665, -0.5681929003180803, 0.21472581804273105, 0.092274717193794112, 0.18181818181818182, 9.7101611508442609, 0.19019818385013582, 0.036175349139890067, 9.1731254285963111, 10.179173916834701, 0.027928082769906368, 0.25000818502714944, 0.00017906797824245588, 0.005990756834485811, -0.0867013703974863, -0.40848348263381595, 0.18924480349138034
# 4, 0.042239707000000008, 0.0, 2.7803984030000004, 0.12577493507676857, 0.01581933429356535, 2.4995465000000001, 3.0909145000000002, 0.04413940559456158, 0.31120001704664196, 7.8305704753148489e-05, 0.0034222666585248473, -0.15873691388011282, -0.5420756697796016, 0.1251444803042855, 0.029466618000000028, 0.0, 5.0841591169999996, 0.10072094154314182, 0.010144708065336992, 4.8787846999999998, 5.3432599999999999, -0.017997595257630866, 0.36754569838979856, 5.0216304923418121e-05, 0.0024996061440121778, -0.08577249478269566, -0.43278604188173553, 0.10021607148897636, 0.066283482000000046, 0.0, 7.7949401150000002, 0.16911205868565465, 0.028598888392900302, 7.4100539999999997, 8.1229279999999999, 0.039385416704700318, 0.25402588942456661, 0.00014156449754485636, 0.0052627424109967721, -0.2811272022567255, -0.6018222030596649, 0.16826437385546383, 0.062971847213146429, 0.16161616161616163, 9.7142691868598323, 0.16627854652850019, 0.027648555035630601, 9.330438777993459, 9.9956296506995237, 0.018933639395131933, 0.26874403190902973, 0.00013686034742637138, 0.0053315146933474522, -0.5639320231297624, -0.3486350637397546, 0.16544506485620625
# 4, 0.017316066000000001, 0.0, 2.7760050519999999, 0.12260354437735269, 0.015031629093889488, 2.4995465000000001, 2.9682117000000003, 0.014028518402699692, 0.36901395113779545, 0.00014730996512011684, 0.0049292797808652977, -0.47220142076860444, -0.5422952866761075, 0.12137131667742465, 0.012102678000000013, 0.0, 5.0913536920000002, 0.060362171896166772, 0.0036435917960223854, 4.9727569999999996, 5.1954180000000001, -0.001980483734712545, 0.2847218234390777, 3.5707199601019383e-05, 0.00093322022511374515, 0.0014085111865007032, -0.7802576091126054, 0.05975550150489859, 0.032609022999999994, 0.0, 7.744775638000001, 0.16005118303675114, 0.02561638119146362, 7.4669165999999993, 8.0828249999999997, 0.0019318708099747991, 0.30003154228971313, 0.00025104053567634346, 0.0068562279784775094, -0.1580524536241806, -0.9125091877958558, 0.15844258760710248, 0.030742361986472844, 0.10101010101010101, 9.6760951199848453, 0.1635859250441129, 0.026760354872538129, 9.3377489404826264, 9.9956296506995237, 0.0031103183647639798, 0.29843050900324686, 0.00026225147775087371, 0.0071194791553835174, -0.48573203376873336, -0.49785943696251733, 0.16194180366751312

data_set = np.loadtxt('Dataset_Erick.csv', delimiter=",")
x_train = data_set[:, 1:]                      # 2D Array = mengambil data dari Dataset_Erick dimulai dari index 1 sampai ke belakang
labels = data_set[:, 0]                        # 1D Array = mengambil data dari Dataset_Erick hanya mengambil index 0

In [14]:
CLASSES = 5                                    # Kita ada 5 aktivitas (0 = STANDING, 1 = WALKING, 2 = RUNNING, 3 = STAIRS, 4 = ON_TRAIN)
y_train = np.zeros([len(labels), CLASSES])     # Kita membuat array 2D dengan size n x m (n = banyak dataset, m = banyak aktivitas) dan diisi dengan angka 0

In [15]:
# Setiap dataset, index aktivitas diberi angka 1 

for i in range(len(labels)):                   
    y_train[i][int(labels[i])] = 1

In [16]:
INPUT_LENGHT = x_train.shape[1] # ambil panjang array 
HIDDEN_UNITS = 1000

hidden_input = np.random.normal(size=[INPUT_LENGHT, HIDDEN_UNITS]) # random data dengan array 2D dengan size INPUT_LENGHT x HIDDEN_UNITS

In [17]:
def input_to_hidden(x):
    a = np.dot(x, hidden_input)     # perkalian matrix
    a = np.maximum(a, 0, a)         # ReLU
    return a

# Referensi ReLU:
# 1. https://en.wikipedia.org/wiki/Rectifier_(neural_networks)

# 2. https://www.kaggle.com/dansbecker/rectified-linear-units-relu-in-deep-learning

In [18]:
# Rumus:
# https://www.sciencedirect.com/science/article/pii/S0893608014000203
# Page 3 Sub-title 3.3 RKELM-based recognition model construction
#
# NB: Ini hanya ELM, sehingga kita akan membuang (1 / lambda) pada rumus

K = input_to_hidden(x_train)
Kt = np.transpose(K)

hasil_1 = np.linalg.pinv(np.dot(Kt, K)) # (Kt x K) ^ -1
hasil_2 = np.dot(Kt, y_train)           # (Kt x T)
B = np.dot(hasil_1, hasil_2)

In [19]:
# Memulai mengklasifikasi

def predict(x):
    x = input_to_hidden(x)
    y = np.dot(x, B)
    return y

p = predict(data_test) # data_test diambil di Line In [11]

correct = 0
total = p.shape[0]
clasification = np.zeros([total])

for i in range(total):
    clasification[i] = np.argmax(p[i])

In [20]:
# Cetak Hasil Klasifikasi
clasification

array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 0., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 4.])

In [21]:
# Beberapa link tentang fungsi2 yang digunakan:
# 1. np.dot (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.dot.html)
# 2. np.std (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.std.html)
# 3. np.zeros (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.zeros.html)
# 4. np.argmax (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.argmax.html)
# 5. np.loadtxt (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.loadtxt.html)
# 6. np.maximum (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.maximum.html)
# 7. np.transpose (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.transpose.html)
# 8. np.linalg.pinv (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.linalg.pinv.html)
# 9. np.random.normal (https://docs.scipy.org/doc/numpy-1.15.0/reference/generated/numpy.random.normal.html)